<a href="https://colab.research.google.com/github/oeunji/deepLerning-ML/blob/main/HW2_2022136087_%EC%9D%B4%EC%9D%80%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## calss TitanicDataset(Dataset)
- PyTorch의 Dataset을 상속받아 타이타닉 데이터를 텐서로 관리하는 기능 구현
- 입력 피쳐 x, 타겟 lable의 y를 받아, 데이터셋의 샘플을 하나씩 접근할 수 있는 입력 메서드 정의

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

class TitanicDataset(Dataset):
  def __init__(self, X, y):
    # 데이터셋의 입력 피쳐와 타겟 레이블을 파이토치 텐서로 변환함.
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    # 데이터셋의 전체 샘플 수 반환.
    return len(self.X)

  def __getitem__(self, idx):
    # 주어진 인덱스에 해당하는 샘플 반환.
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    # 데이터셋의 크기, 입력, 타겟의 shape의 정보 반환.
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

## class Titanic TestDataset(Dataset)
테스트 데이터셋을 위한 클래스. 테스트 데이터는 타겟 값이 없어 입력 피쳐 값만 관리함.

In [ ]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    # 테스트 데이터는 타겟 값이 없어 피쳐만 저장함.
    self.X = torch.FloatTensor(X)

  def __len__(self):
    # 데이터셋의 전체 샘플 수 반환.
    return len(self.X)

  def __getitem__(self, idx):
    # 주어진 인덱스에 해당하는 샘플의 피쳐를 반환.
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    # 데이터셋의 크기와 입력 피쳐의 shapte
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

## def get_preprocessed_dataset()
- 이 함수는 타이타닉 데이터를 불러와서 전처리한 후에 학습, 검증, 테스트 데이터셋으로 나눔.
- 각 전처리 단계는 이후 정의된 전처리 함수들에서 처리함.

In [ ]:
def get_preprocessed_dataset():
    #CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
    CURRENT_FILE_PATH = os.getcwd()  # 주피터 환경에 최적화하기 위해 수정. 현재 작업 디렉토리를 사용

    # 타이타닉 데이터 로드
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # 학습 및 테스트 데이터를 하나의 데이터 프레임으로 합쳐서 전처리함.
    all_df = pd.concat([train_df, test_df], sort = False)

    # 각 전처리 단계 함수를 호출함.
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # 타겟 값이 있는 행은 학습 데이터, 없는 행은 테스트 데이터로 분리함.
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # PyTorch Dataset으로 변환함.
    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

## def get_preprocessed_dataset_1(all_df) ~ 6
전처리 함수

In [ ]:
def get_preprocessed_dataset_1(all_df):
    # 전처리 함수
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

In [ ]:
def get_preprocessed_dataset_2(all_df):
    # 전처리 함수
    # name을 세 개의 컬럼(family_name, honorific, name으로 분리하여 다시 all_df에 합침)
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

In [ ]:
def get_preprocessed_dataset_3(all_df):
    # 전처리 함수
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

In [ ]:
def get_preprocessed_dataset_4(all_df):
    # 전처리 함수
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    #all_df["alone"].fillna(0, inplace=True)
    all_df["alone"] = all_df["alone"].fillna(0) # 경고문을 방지하기 위해 수정함.

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

In [ ]:
def get_preprocessed_dataset_5(all_df):
    # 전처리 함수
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    #all_df["Embarked"].fillna("missing", inplace=True)
    all_df["Embarked"] = all_df["Embarked"].fillna("missing") # 경고문을 방지하기 위해 수정함.

    return all_df

In [ ]:
def get_preprocessed_dataset_6(all_df):
    # 전처리 함수
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

## class MyModel(nn.Module)
- MyModel은 신경망 모델 클래스로 torch.nn.Module을 상속 받아 사용함.
- 입력 크기가 n_input인 3개의 Ninear 레이어와 활성화 함수인 ReLu를 사용함.

In [ ]:
from torch import nn
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, 30),
      nn.ReLU(),
      nn.Linear(30, 30),
      nn.ReLU(),
      nn.Linear(30, n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

## def test(test_data_loader)
- test 함수는 테스트 데이터를 모델에 통과시켜 예측 결과를 출력하는 역할임.
- test_data_loader에서 배치 하나를 가져와서 입력 데이터를 신경망 모델에 통과하고 출력 값을 얻음.
- 출력 값에서 가장 높은 확률을 가진 클레스를 예측하고, 해당 결과를 892번부터 출력함.

In [ ]:
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader))    # 테스트 데이터에서 배치를 하나 가져옴
  print("{0}".format(batch['input'].shape)) # 배치 입력 데이터의 형태 출력
  my_model = MyModel(n_input=11, n_output=2) # 11개의 입력과 2개의 출력을 갖는 모델 생성
  output_batch = my_model(batch['input']) # 모델에 입력 데이터를 넣고 예측값을 얻음
  prediction_batch = torch.argmax(output_batch, dim=1) # 각 예측값에서 가장 높은 확률의 클래스 예측
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item()) # 892번부터 예측값 출력

## if __name__ == "__main__":
- 전처리된 train, validation, test 데이터셋을 생성하여, 이를 DataLoader에 넣어 학습과 테스트를 준비함.
- train_data_loader와 validation_data_loader는 각각 학습과 검증을 위한 데이터셋 로더.
- 배치 크기는 16
- 학습과 검증 데이터의 배치 출력
- test_data_loader를 통해 테스트 데이터를 사용하여 예측 결과 출력.

In [ ]:
if __name__ == "__main__":
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  test(test_data_loader)

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 2.0000,  1.0000, 32.0000,  2.0000,  0.0000, 73.5000,  2.0000, 21.1792,
         2.0000,  2.0000,  0.0000]): 0
1 - tensor([ 3.0000,  1.0000, 17.0000,  0.0000,  0.0000,  7.1250,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 1.0000,  1.0000, 30.0000,  0.0000,  0.0000, 27.7500,  0.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 1.0000,  0.0000, 17.0000,  1.0000,  0.0000, 57.0000,  2.0000, 87.5090,
         3.0000,  1.0000,  0.0000]): 1
4 - tensor([ 3.0000,  1.0000, 36.0000,  0.0000,  0.0000,  7.4958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 1.0000,  1.0000, 29.0000,  0.0000,  0.0000, 42.4000,  2.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 1.0000,  1.0000, 50.0000,  1.0000,  0.0000, 55.9000,  2.0000, 87.5090,
         2.0000,  1.0000,  0.0000]): 0
7 - tensor([ 1.00